# NDWI Analysis - NL dams

( Description )

## Import libraries

In [1]:
pip install pymannkendall


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip3.10 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandarallel


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip3.10 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
import planetary_computer as pc
import geopandas as gpd
import h3
import pandas as pd

import rasterio
from rasterio import windows
from rasterio import features
from rasterio import warp
import rasterio.mask
from rasterio.enums import Resampling
from rasterio.merge import merge

import numpy as np
from PIL import Image

import matplotlib.pyplot as plt

from shapely.geometry import Point

from tqdm import tqdm

import os
import sys
module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/usr/local/python/lib/python3.10/site-packages/osmnx/utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


## Config notebook

In [2]:
# Set raster spectral bands and analysis equation
band_name_dict = {'green':[False], #If GSD(resolution) of band is different, set True.
                  'nir':[False], #If GSD(resolution) of band is different, set True.
                  'eq':["(green-nir)/(green+nir)"]}

# Set analysis name
index_analysis = 'ndwi'

# Set directory to save analysed rasters
tmp_dir = f'../../../data/processed/tmp_{index_analysis}/'

# Set desired hex resolution
res = [11]

# Set frequency of search for rasters (MS = Month Start)
freq = 'MS'

# Set start and end date for search (Sentinel-2 2A has images since mids 2015)
start_date = '2016-01-01'
end_date = '2022-12-31'

# Save?
save = True # True

# Del rasters after processing.
del_data = False # True

# city can be substituted by "place". (presa_laboca)
city = 'presa_elcuchillo'

#Set filter if necessary. Defaults to "{}". Example: No images with cloud cover avobe 10%: {"eo:cloud_cover": {"lt": 10}}
query = {"eo:cloud_cover": {"lt": 10}}

# Set satellite. Defaults to "sentinel-2-l2a"
satellite = 'sentinel-2-l2a'

## Download data

### Download data - area of interest

In [3]:
#Load data
presa_original = gpd.read_file("../../../data/external/temporal_todocker/presa_elcuchillo.gpkg")

Area of interest treatment

In [4]:
#Filter for relevant data
columns_tokeep = ['Name','geometry']
presa = presa_original[columns_tokeep]

#Create buffer for dam geometry
polygon = presa.to_crs("EPSG:6372").buffer(500)
polygon = polygon.to_crs("EPSG:4326")
polygon = gpd.GeoDataFrame(geometry=polygon).dissolve().geometry

#Review result
print(polygon.shape)
polygon.head(2)

(1,)


0    POLYGON ((-99.36587 25.66869, -99.36729 25.674...
Name: geometry, dtype: geometry

### Data download - Download and proccess rasters

In [5]:
#download_raster_from_pc(gdf, index_analysis, city, freq, start_date, end_date, tmp_dir, band_name_dict, query={}, satellite="sentinel-2-l2a"):
df_len = aup.download_raster_from_pc(polygon, index_analysis, city, freq, start_date, end_date, tmp_dir, band_name_dict, query = query, satellite = satellite)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 84/84 [00:00<00:00, 270.97it/s]


## Debug paso por paso

In [15]:
#download_raster_from_pc(gdf, index_analysis, city, freq, start_date, end_date, tmp_dir, band_name_dict, query={}, satellite="sentinel-2-l2a"):
gdf = polygon.copy()

In [16]:
poly = gdf.to_crs("EPSG:6372").buffer(500)
poly = poly.to_crs("EPSG:4326")
poly = gpd.GeoDataFrame(geometry=poly).dissolve().geometry

# Extracts coordinates from polygon as DataFrame
coord_val = poly.bounds
# Gets coordinates for bounding box
n = coord_val.maxy.max()
s = coord_val.miny.min()
e = coord_val.maxx.max()
w = coord_val.minx.min()

# Sets the coordinates for the area of interest
area_of_interest = {
    "type": "Polygon",
    "coordinates": [
        [
            [e, s],
            [w, s],
            [w, n],
            [e, n],
            [e, s],
        ]
    ],
}

# create time of interest
time_of_interest = aup.create_time_of_interest(start_date, end_date, freq=freq)

# gathers items for time and area of interest
items = aup.gather_items(time_of_interest, area_of_interest, query=query, satellite=satellite)

date_list = aup.available_datasets(items, satellite)

# create dictionary from links
band_name_list = list(band_name_dict.keys())[:-1]
assets_hrefs = aup.link_dict(band_name_list, items, date_list)

# analyze available data according to raster properties
df_len, missing_months = aup.df_date_links(assets_hrefs, start_date, end_date, 
                                       band_name_list, freq)
aup.available_data_check(df_len, missing_months) # test for missing months

print(missing_months)

21


In [17]:
# creates raster and analyzes percentage of missing data points
df_len, missing_months = aup.df_date_links(assets_hrefs, start_date, end_date, 
                                       band_name_list, freq)
pct_missing = round(missing_months/len(df_len),2)*100
pct_missing

25.0

In [18]:
# raster cropping with bounding box from earlier 
bounding_box = gpd.GeoDataFrame(geometry=poly).envelope
gdf_bb = gpd.GeoDataFrame(gpd.GeoSeries(bounding_box), columns=['geometry'])

# create GeoDataFrame to test nan values in raster
gdf_raster_test = gdf.to_crs("EPSG:6372").buffer(1)
gdf_raster_test = gdf_raster_test.to_crs("EPSG:4326")
gdf_raster_test = gpd.GeoDataFrame(geometry=gdf_raster_test).dissolve()

In [19]:
# download raster data by month
df_len = aup.create_raster_by_month(
    df_len, index_analysis, city, tmp_dir,
    band_name_dict,date_list, gdf_raster_test,
    gdf_bb, area_of_interest, satellite, query=query)

100%|██████████████████████████████████| 84/84 [00:00<00:00, 360.85it/s]


In [20]:
missing_months = len(df_len.loc[df_len.data_id==0])
missing_months

21

In [41]:
df_len

,year,month,data_id,able_to_download,interpolate
0,2016,1,1,NaN,0
1,2016,2,1,NaN,0
2,2016,3,1,NaN,0
3,2016,4,1,NaN,0
4,2016,5,1,NaN,1
...,...,...,...,...,...
79,2022,8,1,NaN,0
80,2022,9,0,NaN,0
81,2022,10,1,NaN,0
82,2022,11,1,NaN,0


In [42]:
#DEBUG STRATEGY ACCORDING TO ERROR IN aup.raster_interpolation: 
#Allow interpolation in not-found rasters.
df_len.loc[4,'data_id'] = 0 #05/2016
df_len.loc[10,'data_id'] = 0 #11/2016
df_len.loc[17,'data_id'] = 0 #6/2017
df_len.loc[24,'data_id'] = 0 #1/2018
df_len.loc[29,'data_id'] = 0 #6/2018
df_len.loc[39,'data_id'] = 0 #4/2019
df_len.loc[48,'data_id'] = 0 #1/2020
df_len.loc[81,'data_id'] = 0 #10/2022

In [43]:
df_len = aup.raster_interpolation(df_len, city, tmp_dir, index_analysis)

In [44]:
missing_months = len(df_len.loc[df_len.data_id==0])
missing_months

0

## End of debug

### Data download - Create hexgrid from area of interest

In [6]:
hex_gdf = gpd.GeoDataFrame(polygon)
hex_gdf['res'] = res[0]

if len(res)>1:
#If there is more than one resolution
    for r in range(res[0]+1,res[-1]+1): #Skips res 8 because, originally, res 8 is already in original hex_gdf
        hex_tmp = aup.create_hexgrid(polygon, r)
        hex_tmp.rename(columns={f'hex_id_{r}':'hex_id'}, inplace=True)
        hex_tmp['res'] = r
        hex_gdf = pd.concat([hex_gdf, hex_tmp],ignore_index = True, axis = 0)
        del hex_tmp

#If there is only one resolution
else:
    hex_gdf = aup.create_hexgrid(hex_gdf, res[0])
    hex_gdf.rename(columns={f'hex_id_{res[0]}':'hex_id'}, inplace=True)
    hex_gdf['res'] = res[0]

In [7]:
print(hex_gdf.shape)
hex_gdf.head(2)

(83351, 3)


,hex_id,geometry,res
0,8b48b182a249fff,"POLYGON ((-99.30354 25.69848, -99.30335 25.698...",11
1,8b48b1952a01fff,"POLYGON ((-99.33469 25.63062, -99.33450 25.630...",11


## Data processing

### Data processing - Raster to hex

In [8]:
def raster_to_hex_save(hex_gdf_i, df_len, index_analysis, tmp_dir, city, r, save, i=0):
    hex_raster_analysis, df_raster_analysis = aup.raster_to_hex_analysis(hex_gdf_i, df_len, index_analysis,
                                                                tmp_dir, city, r)

    if save:
        hex_raster_analysis.to_file(tmp_dir+'local_save/'+f'{city}_{index_analysis}_HexRes{r}_v{i}.geojson')
        df_raster_analysis.to_csv(tmp_dir+'local_save/'+f'{city}_{index_analysis}_HexRes{r}_v{i}.csv')

In [9]:
for r in list(hex_gdf.res.unique()):

    processing_chunk = 100000

    # filters hexagons at specified resolution
    hex_gdf_res = hex_gdf.loc[hex_gdf.res==r].copy()
    hex_gdf_res = hex_gdf_res.reset_index(drop=True)
    
    if len(hex_gdf_res)>processing_chunk:
        c_processing = len(hex_gdf_res)/processing_chunk
        for i in range(int(c_processing)):
            hex_gdf_i = hex_gdf_res.iloc[int(processing_chunk*i):int(processing_chunk*(1+i))].copy()
            raster_to_hex_save(hex_gdf_i, df_len, index_analysis, tmp_dir, city, r, save, i)
    else:
        hex_gdf_i = hex_gdf_res.copy()
        raster_to_hex_save(hex_gdf_i, df_len, index_analysis, tmp_dir, city, r, save)

 14%|██████████████▌                                                                                       | 1/7 [05:33<33:21, 333.62s/it]

 43%|███████████████████████████████████████████▋                                                          | 3/7 [17:21<23:20, 350.03s/it]

 71%|████████████████████████████████████████████████████████████████████████▊                             | 5/7 [29:31<11:59, 359.62s/it]

  0%|                                                                                                              | 0/12 [08:21<?, ?it/s]


## Data save

In [ ]:
# local save
if save:
    hex_raster_analysis.to_file(tmp_dir+'local_save/'+f'{city}_{index_analysis}_HexRes{r}_.geojson')
    df_raster_analysis.to_csv(tmp_dir+'local_save/'+f'{city}_{index_analysis}_HexRes{r}_.csv')